In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4,5'
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import json
from utils import build_own_prompt
model_path_list = ['/root/.cache/modelscope/hub/qwen/Qwen1___5-7B-Chat', '/root/.cache/modelscope/hub/LLM-Research/Meta-Llama-3-8B-Instruct',
                   '/root/.cache/modelscope/hub/qwen/Qwen1___5-14B-Chat', '/root/.cache/modelscope/hub/ZhipuAI/chatglm3-6b', 
                   '/root/.cache/modelscope/hub/Shanghai_AI_Laboratory/internlm2-chat-7b', '/root/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat',
                   '/root/.cache/modelscope/hub/lockonlvange/autoj-13b-fp16',
                   '/root/.cache/huggingface/hub/models--WeOpenML--PandaLM-7B-v1/snapshots/PandaLM',
                   '/root/finetuned/Qwen1.5-7B-Chat_qlora_epoch_1']

In [2]:
model_name_or_path = model_path_list[-1]
llm = LLM(model=model_name_or_path, tensor_parallel_size=4, trust_remote_code=True, tokenizer_mode="auto", dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=16384)
model_name = model_name_or_path.rsplit('/', 1)[-1]

2024-06-05 01:58:49,065	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-05 01:58:50 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='/root/finetuned/Qwen1.5-7B-Chat_qlora_epoch_1', speculative_config=None, tokenizer='/root/finetuned/Qwen1.5-7B-Chat_qlora_epoch_1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/root/finetuned/Qwen1.5-7B-Chat_qlora_epoch_1)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-05 01:59:00 utils.py:618] Found nccl from library libnccl.so.2
INFO 06-05 01:59:00 pynccl.py:65] vLLM is using nccl==2.20.5
(RayWorkerWrapper pid=760028) INFO 06-05 01:59:00 utils.py:618] Found nccl from library libnccl.so.2
(RayWorkerWrapper pid=760028) INFO 06-05 01:59:00 pynccl.py:65] vLLM is using nccl==2.20.5
WARNING 06-05 01:59:00 custom_all_reduce.py:158] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
(RayWorkerWrapper pid=759870) WARNING 06-05 01:59:00 custom_all_reduce.py:158] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
(RayWorkerWrapper pid=759870) INFO 06-05 01:59:02 model_runner.py:146] Loading model weights took 3.6506 GB
INFO 06-05 01:59:02 model_runner.py:146] Loading model weights took 3.6506 GB
INFO 06-05 01:59:07 distributed_

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(RayWorkerWrapper pid=759870) INFO 06-05 01:59:15 model_runner.py:924] Graph capturing finished in 5 secs.
(RayWorkerWrapper pid=760182) INFO 06-05 01:59:10 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. [repeated 2x across cluster]
(RayWorkerWrapper pid=760182) INFO 06-05 01:59:10 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage. [repeated 2x across cluster]
(RayWorkerWrapper pid=760028) INFO 06-05 01:59:15 model_runner.py:924] Graph capturing finished in 5 secs.


In [3]:
# autoj
texts_autoj = []
input_file_path = '/root/LLM-as-a-judge/datasets/AutoJ/testdata_pairwise.jsonl'
with open(input_file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        prompt = build_own_prompt('autoj') + "\nScenario: " + item['scenario'] + "\nInstruction: " + item['prompt'] + "\nAnswer 1: "+ item['response 1'] + "\nAnswer 2: "+ item['response 2']
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_autoj.append(text)
outputs_autoj = llm.generate(texts_autoj, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(input_file_path, 'r') as fin:
    with open(output_file_path, 'w') as f:
        for output, line in zip(outputs_autoj, fin):
            item = json.loads(line)
            pred_label = output.outputs[0].text
            item['pred_label'] = pred_label
            json.dump(item, f)
            f.write('\n')
            print(pred_label)

Processed prompts: 100%|██████████| 1392/1392 [02:32<00:00,  9.15it/s, Generation Speed: 36.61 toks/s]

[1]
[1]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[0]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[2]
[0]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[1]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[0]
[0]
[2]
[1]
[2]
[0]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[2]


In [4]:
# pandalm
texts_pandalm = []
    
import json
input_file_path = '/root/LLM-as-a-judge/datasets/PandaLM/testset-v1.json'
with open(input_file_path, 'r') as f:
    input_data_pandalm = json.load(f)
    for item in input_data_pandalm:
        prompt = build_own_prompt('pandalm') + "\nInstruction: " + item['instruction'] + "\nInput: " + item['input'] + "\nAnswer 1: " + str(item['response1']) + "\nAnswer 2: "+ str(item['response2'])
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_pandalm.append(text)

outputs_pandalm = llm.generate(texts_pandalm, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_pandalm, input_data_pandalm):
        pred_label = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 999/999 [00:47<00:00, 20.83it/s, Generation Speed: 94.74 toks/s]

[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[0]
[1]
[2]
[1]
[0]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[0]
[2]
[0]
[0]
[2]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[0]
[2]
[0]
[0]
[2]
[0]
[2]
[1]
[2]
[2]
[0]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[0]
[0]
[1]
[0]
[1]
[1]
[2]
[2]
[0]
[2]
[0]
[2]
[1]
[2]
[2]
[0]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[2]
[0]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[2]
[0]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[0]
[2]
[2]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[2]
[0]
[2]
[0]
[1]
[1]
[0]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[0]
[1]


In [5]:
# llmbar_natural
texts_natural = []
input_file_path = '/root/LLM-as-a-judge/datasets/LLMBar/Natural/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_natural = json.load(f)
    for item in input_data_llmbar_natural:
        prompt = build_own_prompt('llmbar') + "\nInstruction: " + item['input'] + "\nAnswer 1: "+ item['output_1'] + "\nAnswer 2: "+ item['output_2']
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_natural.append(text)
outputs_llmbar_natural = llm.generate(texts_natural, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_natural, input_data_llmbar_natural):
        pred_label = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 100/100 [00:04<00:00, 21.95it/s, Generation Speed: 87.81 toks/s]

[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[2]
[2]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[0]
[2]
[2]
[2]


In [6]:
# llmbar_neighbor

texts_neighbor = []
input_file_path = '/root/LLM-as-a-judge/datasets/LLMBar/Adversarial/Neighbor/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_neighbor = json.load(f)
    for item in input_data_llmbar_neighbor:
        prompt = build_own_prompt('llmbar') + "\nInstruction: " + item['input'] + "\nAnswer 1: "+ item['output_1'] + "\nAnswer 2: "+ item['output_2']
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_neighbor.append(text)
outputs_llmbar_neighbor = llm.generate(texts_neighbor, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_neighbor, input_data_llmbar_neighbor):
        pred_label = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 134/134 [00:06<00:00, 21.64it/s, Generation Speed: 86.58 toks/s]

[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[2]
[2]
[0]
[2]
[2]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[1]
[1]
[2]
[1]
[2]
[1]
[2]
[2]


In [7]:
# llmbar_gptinst

texts_gptinst = []
input_file_path = '/root/LLM-as-a-judge/datasets/LLMBar/Adversarial/GPTInst/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_gptinst = json.load(f)
    for item in input_data_llmbar_gptinst:
        prompt = build_own_prompt('llmbar') + "\nInstruction: " + item['input'] + "\nAnswer 1: "+ item['output_1'] + "\nAnswer 2: "+ item['output_2']
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_gptinst.append(text)
outputs_llmbar_gptinst = llm.generate(texts_gptinst, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_gptinst, input_data_llmbar_gptinst):
        pred_label = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 92/92 [00:07<00:00, 12.43it/s, Generation Speed: 49.70 toks/s]

[2]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[1]
[1]
[2]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[2]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[1]
[1]


In [8]:
# llmbar_gptout

texts_gptout = []
input_file_path = '/root/LLM-as-a-judge/datasets/LLMBar/Adversarial/GPTOut/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_gptout = json.load(f)
    for item in input_data_llmbar_gptout:
        prompt = build_own_prompt('llmbar') + "\nInstruction: " + item['input'] + "\nAnswer 1: "+ item['output_1'] + "\nAnswer 2: "+ item['output_2']
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_gptout.append(text)
outputs_llmbar_gptout = llm.generate(texts_gptout, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_gptout, input_data_llmbar_gptout):
        pred_label = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 47/47 [00:01<00:00, 23.98it/s, Generation Speed: 95.91 toks/s]

[2]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[2]
[1]
[2]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[2]


In [9]:
# llmbar_manual

texts_manual = []
input_file_path = '/root/LLM-as-a-judge/datasets/LLMBar/Adversarial/Manual/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_manual = json.load(f)
    for item in input_data_llmbar_manual:
        prompt = build_own_prompt('llmbar') + "\nInstruction: " + item['input'] + "\nAnswer 1: "+ item['output_1'] + "\nAnswer 2: "+ item['output_2']
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_manual.append(text)
outputs_llmbar_manual = llm.generate(texts_manual, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_manual, input_data_llmbar_manual):
        pred_label = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 46/46 [00:03<00:00, 15.07it/s, Generation Speed: 60.29 toks/s]

[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[1]
[2]
[2]
[1]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]


In [10]:
# mtbench
import json
texts_mtbench = []
input_file_path = '/root/LLM-as-a-judge/datasets/MTBench/mt_bench_human.json'
with open(input_file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        prompt = build_own_prompt('mtbench') + 'Turn 1:\n' + 'instruction: \n' + item['conversation_a'][0]['content'] + 'answer1: \n' + item['conversation_a'][1]['content'] + 'answer2: \n' + item['conversation_b'][1]['content'] + 'Turn 2:\n' + 'instruction: \n' + item['conversation_a'][2]['content'] + 'answer1: \n' + item['conversation_a'][3]['content'] + 'answer2: \n' + item['conversation_b'][3]['content']
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_mtbench.append(text)
outputs_mtbench = llm.generate(texts_mtbench, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_own_prompt_response.json'
with open(input_file_path, 'r') as fin:
    with open(output_file_path, 'w') as f:
        for output, line in zip(outputs_mtbench, fin):
            item = json.loads(line)
            pred_label = output.outputs[0].text
            item['pred_label'] = pred_label
            json.dump(item, f)
            f.write('\n')
            print(pred_label)

Processed prompts: 100%|██████████| 3355/3355 [08:21<00:00,  6.69it/s, Generation Speed: 26.76 toks/s]


[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
